<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение-и-тестирование" data-toc-modified-id="Обучение-и-тестирование-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и тестирование</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регресия" data-toc-modified-id="Логистическая-регресия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регресия</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>SGDClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Обзор данных

In [22]:
import numpy as np
import pandas as pd

import re

import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
display(toxic_comments.head(5))

print('-----------------------------------------------------------------')
toxic_comments.info()

print('-----------------------------------------------------------------')

print('Дубликатов -', toxic_comments.duplicated().sum())

print('-----------------------------------------------------------------')

print('Пропусков:')
display(toxic_comments.isna().sum())

print('-----------------------------------------------------------------')

print('Соотношение в целевом признаке:')
display(toxic_comments.toxic.value_counts(normalize=True))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


-----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
-----------------------------------------------------------------
Дубликатов - 0
-----------------------------------------------------------------
Пропусков:


Unnamed: 0    0
text          0
toxic         0
dtype: int64

-----------------------------------------------------------------
Соотношение в целевом признаке:


0    0.898388
1    0.101612
Name: toxic, dtype: float64

**Вывод:**<br>
- В таблице 159 292 объектов. Пропусков нет, явных дубликатов нет
- Тексты комментариев на английском, есть лишние символы, например `\n`
- В целевом признаке ~90% объектов отрицательного класса, то есть в дальнейшем нужно будет учесть это
- Необходимо избавиться от столбца `Unnamed`, так как он фактически дублирует индексы

### Подготовка данных

In [4]:
#удаляем столбец Unnamed:
toxic_comments = toxic_comments.drop(['Unnamed: 0'], axis=1)

In [5]:
#вводим функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [6]:
%%time
#очищаем тексты постов:
toxic_comments['text'] = toxic_comments['text'].apply(clear_text) 

CPU times: user 4.12 s, sys: 27.5 ms, total: 4.15 s
Wall time: 4.15 s


In [7]:
toxic_comments.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [8]:
#вводим функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#вводим функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [9]:
%%time
#леммализируем тексты постов:
toxic_comments['text'] = toxic_comments['text'].apply(lemm_text) 

CPU times: user 17min 13s, sys: 1min 32s, total: 18min 45s
Wall time: 18min 46s


In [10]:
toxic_comments.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [11]:
#разделяем выборки в соотношении 80/10/10:
features = toxic_comments.drop(['toxic'], axis=1) 
target = toxic_comments['toxic']

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=12345)

In [12]:
#глянем размеры выборок:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(15929, 1)
(15929,)
(15930, 1)
(15930,)


In [13]:
#глянем соотношение классов в выборках на примере target_train:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

Доля значений 1 в тренировочной выборке: 0.10166911239631807


In [14]:
#снизим кол-во 0 в выборках train:

toxic_comments_train = toxic_comments.iloc[target_train.index]
target_train_0 = toxic_comments_train[toxic_comments_train['toxic'] == 0]['toxic']
target_train_1 = toxic_comments_train[toxic_comments_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = toxic_comments.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1/0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

Соотношение 1/0 в тренировочной выборке:
0    0.5
1    0.5
Name: toxic, dtype: float64

(25912,)
(25912,)


**Вывод:**<br>
- Удалили ненужный столбец `Unnamed`
- Очистили тексты комментариев от ненужных знаков, леммализировали и убрали стоп-слова
- Сбалансировали данные по значениям классов 

## Обучение и тестирование

In [15]:
features_train = features_train.text

### Логистическая регресия

In [16]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),} 

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 Логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', round(lr_test_f1,2))

F1 Логистической регрессии = 0.77
при параметрах {'lr__C': 10, 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'saga'}

F1 логистической регрессии на валидации = 0.78

финальный F1 логистической регрессии = 0.79
CPU times: user 11min 32s, sys: 7min 54s, total: 19min 26s
Wall time: 19min 27s


### SGDClassifier

In [23]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("clf", SGDClassifier())])
    
parameters = {'clf__loss': ('hinge', 'log', 'modified_huber'),
              'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
              'clf__eta0': (.01, .05, .1, .5),
              'clf__random_state': ([12345])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
sgdc_train_f1 = max(mts)

print('F1 SGDClassifier =', round(sgdc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
sgdc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 SGDClassifier на валидации =', round(sgdc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
sgdc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 SGDClassifier =', round(sgdc_test_f1,2))

F1 SGDClassifier = 0.73
при параметрах {'clf__eta0': 0.05, 'clf__learning_rate': 'constant', 'clf__loss': 'modified_huber', 'clf__random_state': 12345}

F1 SGDClassifier на валидации = 0.73

финальный F1 SGDClassifier = 0.74
CPU times: user 17min 56s, sys: 18 s, total: 18min 14s
Wall time: 18min 19s


### CatBoostClassifier

In [18]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200])} 

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
cbc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 CatBoostClassifier =', round(cbc_test_f1,2))

F1 CatBoostClassifier = 0.74
при параметрах {'cbc__iterations': 200, 'cbc__verbose': False}

F1 CatBoostClassifier на валидации = 0.76

финальный F1 CatBoostClassifier = 0.75
CPU times: user 25min 6s, sys: 5min 19s, total: 30min 26s
Wall time: 30min 34s


## Выводы

In [24]:
#создадим сводную таблицу по показателям F1:
index = ['LogisticRegression',
         'CatBoostClassifier',
         'SGDClassifier',
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    cbc_train_f1,
                                    sgdc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        cbc_valid_f1,
                                        sgdc_valid_f1],
        
        'F1 на тестовой выборке (финальный)': [lr_test_f1,
                                               cbc_test_f1,
                                               sgdc_test_f1]}

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на тестовой выборке (финальный)', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке,F1 на тестовой выборке (финальный)
LogisticRegression,0.767863,0.78153,0.789256
CatBoostClassifier,0.736849,0.75904,0.750541
SGDClassifier,0.731007,0.72900,0.735615


**Вывод:** <br>
- загрузили данные и провели их предобработку - удаление лишних данных, очистку текстов, лемматизацию
- обучили 3 модели с разными гиперпараметрами и выборками и проверили их на тестовой выборке
- выбрали лучшую модель по показателю F1

Аутсайдером среди моделей стал SGDClassifier, так как дал наименьший F1 (0.73 на обучающей и тестовой выборке).

Наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.79. Поскольку требовалось найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75 после обучения, смело рекоммендуем LogisticRegression.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны